# Introduction to BIDS and `pybids`
Brain Imaging Data Structure (**BIDS**) is a a standard for organizing and describing MRI datasets standardised https://bids.neuroimaging.io/
Gorgolewski, K., Auer, T., Calhoun, V. et al. *The brain imaging data structure, a format for organizing and describing outputs of neuroimaging experiments.* Sci Data 3, 160044 (2016). https://doi.org/10.1038/sdata.2016.44

`pybids` is a tool to query, summarize and manipulate data using the BIDS standard. 

**In this tutorial I will illustrate some of the functionality of pybids.layout**

The material is adapted from https://github.com/bids-standard/pybids/tree/master/examples

## Example dataset
When `pybids` is installed, some example datasets are added to the library. They are useful to test the structure of the dataset, but they do not contain the actual image files. 
I will use 'syntetic' dataset for some examples. 

In [1]:
from bids.layout import BIDSLayout
from bids.tests import get_test_data_path
import os

ds_path = os.path.join(get_test_data_path(), 'synthetic')

## The `BIDSLayout`
A `BIDSLayout` is a Python class that represents a BIDS project file tree and provides a variety of helpful methods for querying and manipulating BIDS files. The `BIDSLayout` initializer has a large number of arguments you can use to control the way files are indexed and accessed. But usually you'd initialise a `BIDSLayout` with just its root path location.

In [2]:
# Initialize the layout
layout = BIDSLayout(ds_path)

# Print some basic information about the layout
layout

BIDS Layout: ...ages\bids\tests\data\synthetic | Subjects: 5 | Sessions: 10 | Runs: 10

### Querying the BIDSLayout
The main method for querying `BIDSLayout` is `.get()`.

If we call `.get()` with no additional arguments, we get back a list of all the BIDS files in our dataset.

In [3]:
all_files = layout.get()
print("There are {} files in the layout.".format(len(all_files)))
print("\nThe first 5 files are:")
all_files[:5]

There are 153 files in the layout.

The first 5 files are:


[<BIDSJSONFile filename='D:\miniconda3\lib\site-packages\bids\tests\data\synthetic\dataset_description.json'>,
 <BIDSDataFile filename='D:\miniconda3\lib\site-packages\bids\tests\data\synthetic\participants.tsv'>,
 <BIDSImageFile filename='D:\miniconda3\lib\site-packages\bids\tests\data\synthetic\sub-01\ses-01\anat\sub-01_ses-01_T1w.nii'>,
 <BIDSImageFile filename='D:\miniconda3\lib\site-packages\bids\tests\data\synthetic\sub-01\ses-01\anat\sub-01_ses-01_T1w.nii.gz'>,
 <BIDSImageFile filename='D:\miniconda3\lib\site-packages\bids\tests\data\synthetic\sub-01\ses-01\func\sub-01_ses-01_task-nback_run-01_bold.nii'>]

The returned object is a **Python list**. Each element in the list is a `BIDSFile` object. 

We can also get just filenames.

In [4]:
layout.get(return_type='filename')[:5]

['D:\\miniconda3\\lib\\site-packages\\bids\\tests\\data\\synthetic\\dataset_description.json',
 'D:\\miniconda3\\lib\\site-packages\\bids\\tests\\data\\synthetic\\participants.tsv',
 'D:\\miniconda3\\lib\\site-packages\\bids\\tests\\data\\synthetic\\sub-01\\ses-01\\anat\\sub-01_ses-01_T1w.nii',
 'D:\\miniconda3\\lib\\site-packages\\bids\\tests\\data\\synthetic\\sub-01\\ses-01\\anat\\sub-01_ses-01_T1w.nii.gz',
 'D:\\miniconda3\\lib\\site-packages\\bids\\tests\\data\\synthetic\\sub-01\\ses-01\\func\\sub-01_ses-01_task-nback_run-01_bold.nii']

We can also get such information as
* all `subject` IDs
* all `task` names
* dataset `description`
* the BOLD repetition time tr

In [5]:
layout.get_subjects()

['02', '04', '01', '03', '05']

In [6]:
layout.get_tasks()

['rest', 'nback']

In [7]:
layout.get_dataset_description()

{'Name': 'Synthetic dataset for inclusion in BIDS-examples',
 'BIDSVersion': '1.0.2',
 'License': 'PD',
 'Authors': ['Markiewicz, C. J.']}

In [8]:
layout.get_tr()

2.5

### Filtering files by entities
We can pass any BIDS-defined entities (keywords) to `.get()` method. For example, here's how we would retrieve all BOLD runs with `.nii.gz` extensions for subject `01`.

In [9]:
# Retrieve filenames of all BOLD runs for subject 01
layout.get(subject='01', extension='nii.gz', suffix='bold', return_type='filename')

['D:\\miniconda3\\lib\\site-packages\\bids\\tests\\data\\synthetic\\sub-01\\ses-01\\func\\sub-01_ses-01_task-nback_run-01_bold.nii.gz',
 'D:\\miniconda3\\lib\\site-packages\\bids\\tests\\data\\synthetic\\sub-01\\ses-01\\func\\sub-01_ses-01_task-nback_run-02_bold.nii.gz',
 'D:\\miniconda3\\lib\\site-packages\\bids\\tests\\data\\synthetic\\sub-01\\ses-01\\func\\sub-01_ses-01_task-rest_bold.nii.gz',
 'D:\\miniconda3\\lib\\site-packages\\bids\\tests\\data\\synthetic\\sub-01\\ses-02\\func\\sub-01_ses-02_task-nback_run-01_bold.nii.gz',
 'D:\\miniconda3\\lib\\site-packages\\bids\\tests\\data\\synthetic\\sub-01\\ses-02\\func\\sub-01_ses-02_task-nback_run-02_bold.nii.gz',
 'D:\\miniconda3\\lib\\site-packages\\bids\\tests\\data\\synthetic\\sub-01\\ses-02\\func\\sub-01_ses-02_task-rest_bold.nii.gz']

All of the entities are found in the names of BIDS files. For example *'sub-01_ses-01_task-nback_run-01_bold.nii'* is **subject** **session** **task** **run** **suffix**.

You can get the list of all availabe entities by `layout.get_entities()`.

Here are a few of the most common entities:

* `suffix`: The part of a BIDS filename just before the extension (e.g., 'bold', 'events', 'T1w', etc.).
* `subject`: The subject label
* `session`: The session label
* `run`: The run index
* `task`: The task name

In [10]:
# a list of all awailable entities
layout.get_entities()

{'subject': <bids.layout.models.Entity at 0x1fd83721430>,
 'session': <bids.layout.models.Entity at 0x1fd83721520>,
 'task': <bids.layout.models.Entity at 0x1fd837213a0>,
 'acquisition': <bids.layout.models.Entity at 0x1fd83718580>,
 'ceagent': <bids.layout.models.Entity at 0x1fd83718220>,
 'reconstruction': <bids.layout.models.Entity at 0x1fd83718eb0>,
 'direction': <bids.layout.models.Entity at 0x1fd83718d60>,
 'run': <bids.layout.models.Entity at 0x1fd8373ee20>,
 'proc': <bids.layout.models.Entity at 0x1fd8373e340>,
 'modality': <bids.layout.models.Entity at 0x1fd8373eca0>,
 'echo': <bids.layout.models.Entity at 0x1fd8373eaf0>,
 'flip': <bids.layout.models.Entity at 0x1fd8373e070>,
 'inv': <bids.layout.models.Entity at 0x1fd8373e1c0>,
 'mt': <bids.layout.models.Entity at 0x1fd8373ed00>,
 'part': <bids.layout.models.Entity at 0x1fd8373ec10>,
 'recording': <bids.layout.models.Entity at 0x1fd8373ea90>,
 'space': <bids.layout.models.Entity at 0x1fd8373e370>,
 'suffix': <bids.layout.mode

### Filtering by metadata
Sometimes we want to search for files based not just on their names, but also based on metadata defined in JSON files. We can pass any key that occurs in any JSON file in our project as an argument to `.get()`. We can combine these with any number of core BIDS entities (like `subject`, `run`, etc.).

For example, say we want to retrieve all files where (a) the value of `SamplingFrequency` (a metadata key) is `10`, and (b) the subject is `01` or `02`.

Indeed, we can pass a list for `subject` rather than just a string. This principle applies to all filters: you can always pass in a list instead of a single value, and this will be interpreted as a logical disjunction (i.e., a file must match any one of the provided values).

In [11]:
# Retrieve all files where SamplingFrequency (a metadata key) = 100
# and acquisition = prefrontal, for the first two subjects
layout.get(subject=['01', '02'], SamplingFrequency=10.0)

[<BIDSDataFile filename='D:\miniconda3\lib\site-packages\bids\tests\data\synthetic\sub-01\ses-01\func\sub-01_ses-01_task-nback_run-01_physio.tsv.gz'>,
 <BIDSDataFile filename='D:\miniconda3\lib\site-packages\bids\tests\data\synthetic\sub-01\ses-01\func\sub-01_ses-01_task-nback_run-02_physio.tsv.gz'>,
 <BIDSDataFile filename='D:\miniconda3\lib\site-packages\bids\tests\data\synthetic\sub-01\ses-01\func\sub-01_ses-01_task-rest_physio.tsv.gz'>,
 <BIDSDataFile filename='D:\miniconda3\lib\site-packages\bids\tests\data\synthetic\sub-01\ses-02\func\sub-01_ses-02_task-nback_run-01_physio.tsv.gz'>,
 <BIDSDataFile filename='D:\miniconda3\lib\site-packages\bids\tests\data\synthetic\sub-01\ses-02\func\sub-01_ses-02_task-nback_run-02_physio.tsv.gz'>,
 <BIDSDataFile filename='D:\miniconda3\lib\site-packages\bids\tests\data\synthetic\sub-01\ses-02\func\sub-01_ses-02_task-rest_physio.tsv.gz'>,
 <BIDSDataFile filename='D:\miniconda3\lib\site-packages\bids\tests\data\synthetic\sub-02\ses-01\func\sub-02_s

### Other `return_type` values
We can also ask `get()` to return unique values (or ids) of particular entities. For example, say we want to know which subjects have at least one `T1w` file. We can request that information by setting `return_type='id'`. When using this option, we also need to specify a target entity (or metadata keyword) called `target`. This combination tells the `BIDSLayout` to return the unique values for the specified `target` entity. For example, in the next example, we ask for all of the unique subject IDs that have at least one file with a `T1w` suffix. 

In [12]:
# Ask get() to return the ids of subjects that have T1w files
layout.get(return_type='id', target='subject', suffix='T1w')

['02', '04', '01', '03', '05']

If our `target` is a BIDS entity that corresponds to a particular directory in the BIDS spec (e.g., `subject` or `session`) we can also use `return_type='dir'` to get all matching subdirectories. 

In [13]:
layout.get(return_type='dir', target='subject')

['D:\\miniconda3\\lib\\site-packages\\bids\\tests\\data\\synthetic\\sub-01',
 'D:\\miniconda3\\lib\\site-packages\\bids\\tests\\data\\synthetic\\sub-02',
 'D:\\miniconda3\\lib\\site-packages\\bids\\tests\\data\\synthetic\\sub-03',
 'D:\\miniconda3\\lib\\site-packages\\bids\\tests\\data\\synthetic\\sub-04',
 'D:\\miniconda3\\lib\\site-packages\\bids\\tests\\data\\synthetic\\sub-05']

## The `BIDSFile`
When you call `.get()` on a `BIDSLayout`, the default returned values are objects of class `BIDSFile`. A `BIDSFile` is a lightweight container for individual files in a BIDS dataset. It provides easy access to a variety of useful attributes and methods. Let's take a closer look. First, let's pick a random file from our existing `layout`.

In [14]:
# Pick the 5h file in the dataset
bf = layout.get()[5]

# Print it
bf

<BIDSImageFile filename='D:\miniconda3\lib\site-packages\bids\tests\data\synthetic\sub-01\ses-01\func\sub-01_ses-01_task-nback_run-01_bold.nii.gz'>

Here are some of the attributes and methods available to us in a `BIDSFile` (note that some of these are only available for certain subclasses of `BIDSFile`; e.g., you can't call `get_image()` on a `BIDSFile` that doesn't correspond to an image file!):
* `.path`: The full path of the associated file
* `.filename`: The associated file's filename (without directory)
* `.dirname`: The directory containing the file
* `.get_entities()`: Returns information about entities associated with this `BIDSFile` (optionally including metadata)
* `.get_image()`: Returns the file contents as a nibabel image (only works for image files)
* `.get_df()`: Get file contents as a pandas DataFrame (only works for TSV files)
* `.get_metadata()`: Returns a dictionary of all metadata found in associated JSON files
* `.get_associations()`: Returns a list of all files associated with this one in some way

Let's see some of these in action.

In [15]:
# Print all the entities associated with this file, and their values
bf.get_entities()

{'datatype': 'func',
 'extension': '.nii.gz',
 'run': 1,
 'session': '01',
 'subject': '01',
 'suffix': 'bold',
 'task': 'nback'}

In [16]:
# Print all the metadata associated with this file
bf.get_metadata()

{'RepetitionTime': 2.5, 'TaskName': 'N-Back'}

Here are all the files associated with our target file in some way.

In [17]:
bf.get_associations()

[<BIDSJSONFile filename='D:\miniconda3\lib\site-packages\bids\tests\data\synthetic\task-nback_bold.json'>,
 <BIDSDataFile filename='D:\miniconda3\lib\site-packages\bids\tests\data\synthetic\sub-01\ses-01\func\sub-01_ses-01_task-nback_run-01_physio.tsv.gz'>,
 <BIDSDataFile filename='D:\miniconda3\lib\site-packages\bids\tests\data\synthetic\sub-01\ses-01\func\sub-01_ses-01_task-nback_run-01_stim.tsv.gz'>]

In cases where a file has a `.tsv.gz` or `.tsv` extension, it will automatically be created as a `BIDSDataFile`, and we can easily grab the contents as a pandas `DataFrame`.


In [18]:
# Get the first physiological recording file
recfile = layout.get(suffix='physio')[0]

# Get contents as a DataFrame and show the first few rows
df = recfile.get_df()
df.head()

,onset,respiratory,cardiac
0,0.0,-0.714844,-0.262109
1,0.1,-0.757342,0.048933
2,0.2,-0.796851,0.355185
3,0.3,-0.833215,0.626669
4,0.4,-0.866291,0.836810


## Other utilities

### Exporting a `BIDSLayout` to a pandas `Dataframe`
If you want a summary of all the files in your `BIDSLayout`, but don't want to have to iterate `BIDSFile` objects and extract their entities, you can get a nice bird's-eye view of your dataset using the `to_df()` method.

In [19]:
# Convert the layout to a pandas dataframe
df = layout.to_df()
df.head()

entity,path,datatype,extension,run,scans,session,subject,suffix,task
0,D:\miniconda3\lib\site-packages\bids\tests\dat...,NaN,.json,NaN,NaN,NaN,NaN,description,NaN
1,D:\miniconda3\lib\site-packages\bids\tests\dat...,NaN,.tsv,NaN,NaN,NaN,NaN,participants,NaN
2,D:\miniconda3\lib\site-packages\bids\tests\dat...,anat,.nii,NaN,NaN,01,01,T1w,NaN
3,D:\miniconda3\lib\site-packages\bids\tests\dat...,anat,.nii.gz,NaN,NaN,01,01,T1w,NaN
4,D:\miniconda3\lib\site-packages\bids\tests\dat...,func,.nii,1,NaN,01,01,bold,nback


We can also include metadata in the result if we like (which may blow up our `DataFrame` if we have a large dataset). Note that in this case, most of our cells will have missing values.

In [20]:
df = layout.to_df(metadata=True)
df.head()

entity,path,Columns,RepetitionTime,SamplingFrequency,StartTime,TaskName,datatype,extension,run,scans,session,subject,suffix,task
0,D:\miniconda3\lib\site-packages\bids\tests\dat...,NaN,NaN,NaN,NaN,NaN,NaN,.json,NaN,NaN,NaN,NaN,description,NaN
1,D:\miniconda3\lib\site-packages\bids\tests\dat...,NaN,NaN,NaN,NaN,NaN,NaN,.tsv,NaN,NaN,NaN,NaN,participants,NaN
2,D:\miniconda3\lib\site-packages\bids\tests\dat...,NaN,NaN,NaN,NaN,NaN,anat,.nii,NaN,NaN,01,01,T1w,NaN
3,D:\miniconda3\lib\site-packages\bids\tests\dat...,NaN,NaN,NaN,NaN,NaN,anat,.nii.gz,NaN,NaN,01,01,T1w,NaN
4,D:\miniconda3\lib\site-packages\bids\tests\dat...,NaN,2.5,NaN,NaN,N-Back,func,.nii,1,NaN,01,01,bold,nback


## Retrieving BIDS variables 
BIDS variables are stored in .tsv files at the run, session, subject, or dataset level. You can retrieve these variables with `layout.get_collections()`. The resulting objects can be converted to dataframes and merged with the layout to associate the variables with corresponding scans.

In the following example, we request all subject-level variable data available anywhere in the BIDS project, and merge the results into a single `DataFrame` (by default, we'll get back a single `BIDSVariableCollection` object for each subject). 

In [21]:
# Get subject variables as a dataframe and merge them back in with the layout
subj_df = layout.get_collections(level='subject', merge=True).to_df()
subj_df.head()

,session,subject,systolic_blood_pressure,suffix
0,01,01,112,sessions
1,01,02,114,sessions
2,01,03,112,sessions
3,01,04,111,sessions
4,01,05,114,sessions


In [22]:
# Get dataset variables as a dataframe and merge them back in with the layout
layout.get_collections(level='dataset', merge=True).to_df()

,subject,age,sex,subject_id,suffix
0,01,34,F,001,participants
1,02,38,M,2,participants
2,03,22,M,3,participants
3,04,21,F,4,participants
4,05,42,M,5,participants


## Report generation
`PyBIDS` also allows you to automatically create data acquisition reports based on the available `image` and `meta-data` information. This enables a new level of standardisation and transparency. FAIR-ness, meta-analyses, etc.). 

In [23]:
# import the BIDSReport function from the reports submodule
from bids.reports import BIDSReport

Now we only need to apply the `BIDSReport` function to our `layout` and generate our report. We'll get some warnings as the example data set is missing some parts.

In [24]:
# Initialize a report for the dataset
report = BIDSReport(layout)

In [25]:
# Method generate returns a Counter of unique descriptions across subjects
descriptions = report.generate()

Number of patterns detected: 1
Remember to double-check everything and to replace <deg> with a degree symbol.


As we can see, one pattern was detected and we can evaluate the actual corresponding information, that is our report.

In [26]:
pub_description = descriptions.most_common()[0][0]
print(pub_description)

For session 02:
	MR data were acquired using a UNKNOWN-Tesla MANUFACTURER MODEL MRI scanner.
	Two runs of N-Back UNKNOWN-echo fMRI data were collected (64 slices; repetition time, TR=2500ms; echo time, TE=UNKNOWNms; flip angle, FA=UNKNOWN<deg>; field of view, FOV=128x128mm; matrix size=64x64; voxel size=2x2x2mm). Each run was 2:40 minutes in length, during which 64 functional volumes were acquired. 
	One run of Rest UNKNOWN-echo fMRI data were collected (64 slices; repetition time, TR=2500ms; echo time, TE=UNKNOWNms; flip angle, FA=UNKNOWN<deg>; field of view, FOV=128x128mm; matrix size=64x64; voxel size=2x2x2mm). Each run was 2:40 minutes in length, during which 64 functional volumes were acquired. 

For session 01:
	MR data were acquired using a UNKNOWN-Tesla MANUFACTURER MODEL MRI scanner.
	Two runs of N-Back UNKNOWN-echo fMRI data were collected (64 slices; repetition time, TR=2500ms; echo time, TE=UNKNOWNms; flip angle, FA=UNKNOWN<deg>; field of view, FOV=128x128mm; matrix size=64